# CANCEL TIMES ANALYSES


In [ ]:

append = false;

ccm_neuron_stop_vs_go_pop(subject,projectRoot,projectDate, append)


In [ ]:

% load a list of neurons sessions and units
categoryList = {'presacc','presaccNoVis','presaccRamp','visPresacc'};
categoryList = {'presacc'};

    % Build a new table of the relevant neurons, and a list of the session/Unit
for i = 1 : length(categoryList)

    load(fullfile(dataPath, ['ccm_',categoryList{i},'_neurons']))

    % load the population of cancel time anlysis
    load(fullfile(dataPath, ['ccm_canceled_vs_go_neuronTypes']))

    cancelData = table();
    for i = 1 : size(neurons, 1)
        % find the indices in cancelTypes that correspond to this unit
        iInd = strcmp(neurons.sessionID(i), cancelTypes.sessionID) & strcmp(neurons.unit(i), cancelTypes.unit);
        cancelData = [cancelData; cancelTypes(iInd,:)];


    end




    % How many (among all units X SSDs were go vs stop different during 40 ms peri-SSRT?
    alphaVal = .05;
    peri40msInd = cellfun(@(x) x < alphaVal, cancelData.pValue40msStopStop, 'uni', false);
    notPeri40ms = cellfun(@(x) x >= alphaVal, cancelData.pValue40msStopStop, 'uni', false);

    pPeri40ms = sum(cell2mat(peri40msInd)) / (sum(cell2mat(peri40msInd)) + sum(cell2mat(notPeri40ms)));





    %   START HERE TO USE cancelTime2Std TO SORT NEURONS
    cancelData.cancelTime = cellfun(@(x,y,z) x - y - z, cancelData.cancelTime2Std, cancelData.stopStopSsd, num2cell(cancelData.ssrt), 'uni', false);


    % How many conditions out of all possible canceled?
    allCancelTime = cell2mat(cellfun(@(x) ~isnan(x), cancelData.cancelTime, 'uni', false));
    nCancelCond = sum(cell2mat(cellfun(@(x) ~isnan(x), cancelData.cancelTime, 'uni', false)));
    nTotal = size(cell2mat(cellfun(@(x) ~isnan(x), cancelData.cancelTime, 'uni', false)), 1);
    pCancelCond = nCancelCond / nTotal


    % What's the full distribution of cancel times?
    allCancelTime = cell2mat(cancelData.cancelTime);
    cancelCond = allCancelTime(~isnan(allCancelTime));



    % How many conditions canceled for this neuron?
    nCancelPerNeuron = cellfun(@(x) sum(~isnan(x)), cancelData.cancelTime);
    % How many potential conditions to cancel for this neuron?
    nCondPerNeuron = cellfun(@length, cancelData.cancelTime);

    % If any condition canceled for this neuron, count it as a "cancel" neuron
    neuronsCanceled = nCancelPerNeuron > 0;

    % Probability of a neuron to have at least one condition that cancels
    pNeuronCond = sum(neuronsCanceled) / size(neurons, 1)
    %disp([neurons, num2cell(nCancelPerNeuron)])


    % Which units canceled?
    presaccCancel = neurons(nCancelPerNeuron > 0, :);
    presaccNoCancel = neurons(nCancelPerNeuron == 0, :);

    load(fullfile(dataPath, ['ccm_',categoryList{i},'_neurons']))
    neurons = neurons(nCancelPerNeuron > 0, :);
    save(fullfile(dataPath, ['ccm_',categoryList{i},'Cancel_neurons']))

    load(fullfile(dataPath, ['ccm_',categoryList{i},'_neurons']))
    neurons = neurons(nCancelPerNeuron == 0, :);
    save(fullfile(dataPath, ['ccm_',categoryList{i},'NoCancel_neurons']))

end
            

#Neuronal Separation Distributions

In [ ]:

% load a list of neurons sessions and units
categoryList = {'presacc','presaccNoVis','presaccRamp','visPresacc'};
categoryList = {'presacc'};

% Build a new table of the relevant neurons, and a list of the session/Unit

load(fullfile(dataPath, ['ccm_',categoryList{1},'_neurons']))

% load the population of cancel time anlysis
load(fullfile(dataPath, ['ccm_canceled_vs_go_neuronTypes']))

cancelData = table();
for i = 1 : size(neurons, 1)
    % find the indices in cancelTypes that correspond to this unit
    iInd = strcmp(neurons.sessionID(i), cancelTypes.sessionID) & strcmp(neurons.unit(i), cancelTypes.unit);
    cancelData = [cancelData; cancelTypes(iInd,:)];

end





ssdArray = unique(cell2mat(cancelData.stopStopSsd));
ssdEasy = cell(length(ssdArray), 1);
ssdHard = cell(length(ssdArray), 1);

cancelTimeEasy2Std = cell(length(ssdArray), 1);
cancelTimeHard2Std = cell(length(ssdArray), 1);

cancelTimeEasy6Std = cell(length(ssdArray), 1);
cancelTimeHard6Std = cell(length(ssdArray), 1);
ssdTimeHard6Std = cell(length(ssdArray), 1);

cancelTimeEasySsrt = cell(length(ssdArray), 1);
cancelTimeHardSsrt = cell(length(ssdArray), 1);
ssdTimeHardSsrt = cell(length(ssdArray), 1);


nCol = 2;
nRow = 2;
easyPlot = 1;
hardPlot = 2;

figureHandle = 68;
[axisWidth, axisHeight, xAxesPosition, yAxesPosition] = standard_landscape(nRow, nCol, figureHandle);
clf

ax(1, easyPlot) = axes('units', 'centimeters', 'position', [xAxesPosition(1, easyPlot) yAxesPosition(1, easyPlot) axisWidth axisHeight]);
         hold(ax(1, easyPlot), 'on')
cla
ax(1, hardPlot) = axes('units', 'centimeters', 'position', [xAxesPosition(1, hardPlot) yAxesPosition(1, hardPlot) axisWidth axisHeight]);
         hold(ax(1, hardPlot), 'on')
cla


    easyInd = cellfun(@(x) x == 1, cancelData.stopStopCond, 'uni', false);
    hardInd = cellfun(@(x) x == 2, cancelData.stopStopCond, 'uni', false);

for i = 1 :length(ssdArray)
    ssdInd =  cellfun(@(x) x == ssdArray(i), cancelData.stopStopSsd, 'uni', false);

    iEasyCoh = cellfun(@(x,y,z) x(y & z), cancelData.stopStopCond, easyInd, ssdInd, 'uni', false);
    ssdEasy{i} = cellfun(@(x,y,z) x(y & z), cancelData.stopStopSsd, easyInd, ssdInd, 'uni', false);

    iHardCoh = cellfun(@(x,y,z) x(y & z), cancelData.stopStopCond, hardInd, ssdInd, 'uni', false);
    ssdHard{i} = cellfun(@(x,y,z) x(y & z), cancelData.stopStopSsd, hardInd, ssdInd, 'uni', false);

    fprintf('Easy units: %d\n', sum(~cellfun(@isempty, ssdEasy{i})))
    fprintf('Hard units: %d\n', sum(~cellfun(@isempty, ssdHard{i})))
    if sum(cell2mat(ssdEasy{i}))
        cancelTimeEasy2Std{i} = cellfun(@(x,y,z,k) x(y & z) - k(y & z), cancelData.cancelTime2Std, easyInd, ssdInd, cancelData.stopStopSsd, 'uni', false);
        iUnitEasyInd = cellfun(@(x) ~isempty(x), cancelTimeEasy2Std{i});
        cancelTimeEasySsrt{i} = cell2mat(cancelTimeEasy2Std{i}(iUnitEasyInd)) - cancelData.ssrt(iUnitEasyInd);

        scatter(ax(1, easyPlot), cell2mat(ssdEasy{i}), cell2mat(cancelTimeEasy2Std{i}))
    
    end
    if sum(cell2mat(ssdHard{i}))
        cancelTimeHard2Std{i} = cellfun(@(x,y,z,k) x(y & z) - k(y & z), cancelData.cancelTime2Std, hardInd, ssdInd, cancelData.stopStopSsd, 'uni', false);
        iUnitHardInd = cellfun(@(x) ~isempty(x), cancelTimeHard2Std{i});    
        cancelTimeHardSsrt{i} = cell2mat(cancelTimeHard2Std{i}(iUnitHardInd)) - cancelData.ssrt(iUnitHardInd);

        scatter(ax(1, hardPlot), cell2mat(ssdHard{i}), cell2mat(cancelTimeHard2Std{i}))
    end
    
    
    
%    hard6Std = cellfun(@(x) ~isnan(x), cancelData.cancelTime6Std, 'uni', false);
%    cancelTimeEasy6Std{i} = 



end




#PROCEED TO CHOICE_CANCEL ANALYSES NOTEBOOK